In [10]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../../db/nyt_books.db")

# 1. Row counts for reference.

In [11]:
pd.read_sql_query("SELECT COUNT(*) AS total_books FROM books;", conn)

,total_books
0,1366


In [12]:
pd.read_sql_query("SELECT COUNT(*) AS total_appearances FROM appearances;", conn)

,total_appearances
0,5775


# 2. Most frequent authors on the bestsellers list?

In [13]:
query = """
SELECT 
    b.author,
    COUNT(*) AS total_weeks,
    COUNT(DISTINCT a.list_date) AS unique_weeks,
    COUNT(DISTINCT b.title) AS unique_titles
FROM appearances a
JOIN books b ON a.isbn13 = b.isbn13
GROUP BY b.author
ORDER BY total_weeks DESC
LIMIT 20;
"""
pd.read_sql_query(query, conn)

,author,total_weeks,unique_weeks,unique_titles
0,Rebecca Yarros,219,128,4
1,John Grisham,172,172,11
2,Stephen King,163,161,10
3,Delia Owens,134,134,1
4,David Baldacci,134,134,17
5,Danielle Steel,126,126,50
6,Matt Haig,111,111,2
7,Kristin Hannah,102,97,4
8,Bonnie Garmus,99,99,1
9,Nicholas Sparks,89,89,5


# 3. Which publishers dominate the fiction bestsellers list?

In [14]:
query = """
SELECT
    b.publisher,
    COUNT(*) AS total_weeks,
    COUNT(DISTINCT b.title) AS unique_titles
FROM appearances a
JOIN books b ON a.isbn13 = b.isbn13
GROUP BY b.publisher
ORDER BY total_weeks DESC;
"""
pd.read_sql_query(query, conn)


,publisher,total_weeks,unique_titles
0,"Little, Brown",483,95
1,Doubleday,384,29
2,Putnam,357,90
3,Grand Central,305,65
4,St. Martin's,264,68
...,...,...,...
115,Cemetery Dance,1,1
116,Baen,1,1
117,Avid Reader/Simon & Schuster,1,1
118,Amara,1,1


# 4. Which books had the greatest longevity on the list?

In [15]:
query = """
SELECT
    b.title,
    b.author,
    MAX(a.weeks_on_list) AS peak_weeks
FROM appearances a
JOIN books b ON a.isbn13 = b.isbn13
GROUP BY b.isbn13
ORDER BY peak_weeks DESC
LIMIT 20;
"""
pd.read_sql_query(query, conn)


,title,author,peak_weeks
0,WHERE THE CRAWDADS SING,Delia Owens,134
1,THE MIDNIGHT LIBRARY,Matt Haig,106
2,LESSONS IN CHEMISTRY,Bonnie Garmus,99
3,FOURTH WING,Rebecca Yarros,98
4,FOURTH WING,Rebecca Yarros,94
5,THE NIGHTINGALE,Kristin Hannah,87
6,IRON FLAME,Rebecca Yarros,82
7,IRON FLAME,Rebecca Yarros,79
8,DEMON COPPERHEAD,Barbara Kingsolver,70
9,THE WOMEN,Kristin Hannah,66


# 5. Which books hit #1 the fastest after debut?

In [16]:
query = """
WITH ranked AS (
    SELECT
        b.title,
        b.author,
        a.list_date,
        a.rank,
        ROW_NUMBER() OVER (PARTITION BY b.isbn13 ORDER BY a.list_date) AS first_week
    FROM appearances a
    JOIN books b ON a.isbn13 = b.isbn13
)
SELECT
    title,
    author,
    list_date AS date_hit_1,
    first_week AS week_number
FROM ranked
WHERE rank = 1
ORDER BY week_number ASC
LIMIT 20;
"""
pd.read_sql_query(query, conn)


,title,author,date_hit_1,week_number
0,THE NEXT PERSON YOU MEET IN HEAVEN,Mitch Albom,2018-10-28,1
1,HEAT 2,Michael Mann and Meg Gardiner,2022-08-28,1
2,THE OTHER WOMAN,Daniel Silva,2018-08-05,1
3,THE NEW GIRL,Daniel Silva,2019-08-04,1
4,THE ORDER,Daniel Silva,2020-08-02,1
5,PORTRAIT OF AN UNKNOWN WOMAN,Daniel Silva,2022-08-07,1
6,THE CELLIST,Daniel Silva,2021-08-01,1
7,THE STRANGER IN THE LIFEBOAT,Mitch Albom,2021-11-21,1
8,THE PARIS APARTMENT,Lucy Foley,2022-03-13,1
9,BABEL,R.F. Kuang,2022-09-11,1


# 6. Which titles had the most volatile rank changes week-to-week?

In [17]:
query = """
WITH ordered AS (
    SELECT
        a.isbn13,
        b.title,
        b.author,
        a.list_date,
        a.rank,
        LAG(a.rank) OVER (PARTITION BY a.isbn13 ORDER BY a.list_date) AS prev_rank
    FROM appearances a
    JOIN books b ON a.isbn13 = b.isbn13
)
SELECT
    title,
    author,
    list_date,
    rank,
    prev_rank,
    (prev_rank - rank) AS rank_change
FROM ordered
WHERE prev_rank IS NOT NULL
ORDER BY ABS(rank_change) DESC
LIMIT 20;
"""
pd.read_sql_query(query, conn)


,title,author,list_date,rank,prev_rank,rank_change
0,HEAT 2,Michael Mann and Meg Gardiner,2022-09-04,14,1,-13
1,BABEL,R.F. Kuang,2023-01-01,14,1,-13
2,THE BOOK OF ELSEWHERE,Keanu Reeves and China Miéville,2024-08-18,15,2,-13
3,FOR RICHER FOR POORER,Danielle Steel,2025-09-07,14,1,-13
4,THE IRRESISTIBLE URGE TO FALL FOR YOUR ENEMY,Brigitte Knightley,2025-08-03,15,2,-13
5,THE BUTCHER GAME,Alaina Urquhart,2024-10-13,14,1,-13
6,ONLY THE DEAD,Jack Carr,2023-06-11,14,1,-13
7,EVVIE DRAKE STARTS OVER,Linda Holmes,2019-07-21,3,15,12
8,BLACK CAKE,Charmaine Wilkerson,2022-02-27,14,2,-12
9,A MOTHER'S LOVE,Danielle Steel,2025-07-20,15,3,-12


# 7. What seasonal trends exist in new book entries? When are most bestsellers debuting?

In [18]:
query = """
SELECT
    STRFTIME('%m', list_date) AS month,
    COUNT(*) AS new_entries
FROM (
    SELECT
        isbn13,
        MIN(list_date) AS list_date
    FROM appearances
    GROUP BY isbn13
)
GROUP BY month
ORDER BY month;
"""
pd.read_sql_query(query, conn)


,month,new_entries
0,01,55
1,02,82
2,03,125
3,04,106
4,05,124
5,06,119
6,07,108
7,08,111
8,09,155
9,10,183


# 8. What yearly trends exist in average weeks on the list? Is the market getting more competitive?

In [19]:
query = """
SELECT
    STRFTIME('%Y', list_date) AS year,
    AVG(weeks_on_list) AS avg_weeks
FROM appearances
GROUP BY year
ORDER BY year;
"""
pd.read_sql_query(query, conn)

,year,avg_weeks
0,2018,7.669333
1,2019,7.312821
2,2020,13.784615
3,2021,11.206410
4,2022,11.217949
5,2023,14.532075
6,2024,15.302564
7,2025,15.809929


# 9. What is market turnover like? How many new books enter the list each year?

In [20]:
query = """
SELECT
    STRFTIME('%Y', MIN(list_date)) AS year,
    COUNT(*) AS new_books
FROM appearances a
GROUP BY isbn13
ORDER BY year;
"""
pd.read_sql_query(query, conn)



,year,new_books
0,2018,12
1,2018,3
2,2018,1
3,2018,1
4,2018,1
...,...,...
1361,2025,1
1362,2025,1
1363,2025,4
1364,2025,1


# 10. Which books debut the highest on the list?

In [21]:
query = """
WITH debut AS (
    SELECT
        b.title,
        b.author,
        MIN(a.list_date) AS debut_date,
        MIN(a.rank) AS debut_rank
    FROM appearances a
    JOIN books b ON a.isbn13 = b.isbn13
    GROUP BY b.isbn13
)
SELECT *
FROM debut
ORDER BY debut_rank ASC
LIMIT 20;
"""
pd.read_sql_query(query, conn)


,title,author,debut_date,debut_rank
0,THE NEXT PERSON YOU MEET IN HEAVEN,Mitch Albom,2018-10-28,1
1,HEAT 2,Michael Mann and Meg Gardiner,2022-08-28,1
2,THE OTHER WOMAN,Daniel Silva,2018-08-05,1
3,THE NEW GIRL,Daniel Silva,2019-08-04,1
4,THE ORDER,Daniel Silva,2020-08-02,1
5,PORTRAIT OF AN UNKNOWN WOMAN,Daniel Silva,2022-08-07,1
6,THE CELLIST,Daniel Silva,2021-08-01,1
7,THE STRANGER IN THE LIFEBOAT,Mitch Albom,2021-11-21,1
8,THE PARIS APARTMENT,Lucy Foley,2022-03-13,1
9,BABEL,R.F. Kuang,2022-09-11,1


# 11. Which authors had repeat success on the list?

In [22]:
query = """
SELECT
    author,
    COUNT(DISTINCT title) AS total_titles,
    SUM(weeks_on_list) AS total_weeks
FROM books b
JOIN appearances a ON a.isbn13 = b.isbn13
GROUP BY author
HAVING total_titles >= 2
ORDER BY total_weeks DESC;
"""
pd.read_sql_query(query, conn)


,author,total_titles,total_weeks
0,Rebecca Yarros,4,8996
1,Matt Haig,2,5686
2,Kristin Hannah,4,3092
3,Barbara Kingsolver,2,2506
4,Laura Dave,2,2148
...,...,...,...
244,Chris Bohjalian,2,2
245,Brent Weeks,2,2
246,Blake Crouch,2,2
247,Beatriz Williams,2,2


# 12. Which books had the most consistent popularity (least rank fluctuation)?

In [23]:
query = """
WITH diffs AS (
    SELECT
        a.isbn13,
        b.title,
        b.author,
        a.list_date,
        a.rank,
        LAG(rank) OVER (PARTITION BY a.isbn13 ORDER BY a.list_date) AS prev
    FROM appearances a
    JOIN books b ON a.isbn13 = b.isbn13
)
SELECT
    title,
    author,
    AVG(ABS(rank - prev)) AS avg_rank_change
FROM diffs
WHERE prev IS NOT NULL
GROUP BY isbn13
ORDER BY avg_rank_change ASC
LIMIT 20;
"""
pd.read_sql_query(query, conn)


,title,author,avg_rank_change
0,THE NIGHT WATCHMAN,Louise Erdrich,0.000000
1,EXHALATION,Ted Chiang,0.000000
2,THE DEATH OF MRS. WESTAWAY,Ruth Ware,0.000000
3,THE HOUSE OF EVE,Sadeqa Johnson,0.000000
4,WICKED,Gregory Maguire,0.333333
5,THE LAST HOUSE GUEST,Megan Miranda,0.333333
6,WILD DARK SHORE,Charlotte McConaghy,0.500000
7,FIRST LIE WINS,Ashley Elston,0.800000
8,"RUN, ROSE, RUN",Dolly Parton and James Patterson,0.875000
9,HAPPY PLACE,Emily Henry,0.888889
